In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
# 数据准备

In [3]:
(x,y),(x_test,y_test) = keras.datasets.mnist.load_data()

In [4]:
def proccess(x,y):
    x = tf.convert_to_tensor(tf.cast(x,dtype=tf.float32))/255.
    y = tf.convert_to_tensor(tf.cast(y,dtype=tf.int32))
    x = tf.reshape(x,[28*28])
    y = tf.one_hot(y,depth=10)
    return x,y

In [5]:
# 设置banchsize

In [6]:
banchsize = 128

In [7]:
train_db = tf.data.Dataset.from_tensor_slices((x,y)).map(proccess).shuffle(10000).batch(banchsize)
test_db = tf.data.Dataset.from_tensor_slices((x,y)).map(proccess).batch(banchsize)

In [8]:
network = tf.keras.Sequential([
    tf.keras.layers.Dense(256,activation='relu'), # [b,784] =>[b,256]
    tf.keras.layers.Dense(128,activation='relu'), # [b,256] =>[b,128]
    tf.keras.layers.Dense(64,activation='relu'), # [b,128] =>[b,64]
    tf.keras.layers.Dense(32,activation='relu'), # [b,64] =>[b,32]
    tf.keras.layers.Dense(10), # [b,32] =>[b,10]
])

In [9]:
network.build(input_shape=[None,28*28])

In [10]:
network.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               200960    
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 32)                2080      
                                                                 
 dense_4 (Dense)             (None, 10)                330       
                                                                 
Total params: 244,522
Trainable params: 244,522
Non-trainable params: 0
_________________________________________________________________


In [13]:
network.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
               loss=tf.losses.CategoricalCrossentropy(from_logits=True),
            metrics=['accuracy']) # metrics 置信度函数

In [14]:
network.fit(train_db,epochs=10,validation_data=test_db,validation_freq=2)

Epoch 1/10
469/469 [==============================] - 3s 4ms/step - loss: 0.0191 - accuracy: 0.9937
Epoch 2/10
469/469 [==============================] - 3s 6ms/step - loss: 0.0153 - accuracy: 0.9952 - val_loss: 0.0087 - val_accuracy: 0.9970
Epoch 3/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0162 - accuracy: 0.9948
Epoch 4/10
469/469 [==============================] - 3s 6ms/step - loss: 0.0112 - accuracy: 0.9963 - val_loss: 0.0126 - val_accuracy: 0.9960
Epoch 5/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0118 - accuracy: 0.9961
Epoch 6/10
469/469 [==============================] - 3s 6ms/step - loss: 0.0139 - accuracy: 0.9955 - val_loss: 0.0050 - val_accuracy: 0.9984
Epoch 7/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0105 - accuracy: 0.9964
Epoch 8/10
469/469 [==============================] - 3s 6ms/step - loss: 0.0105 - accuracy: 0.9965 - val_loss: 0.0091 - val_accuracy: 0.9971
Epoch 9/10
469/469 [============

In [16]:
sample = next(iter(test_db))

In [20]:
x = sample[0]
y = sample[1] # one-hot
pred = network.predict(x) # [b, 10]获得一个batch的值  # 作预测
# convert back to number 
y = tf.argmax(y, axis=1)
pred = tf.argmax(pred, axis=1)

4/4 [==============================] - 0s 3ms/step


In [21]:
print(pred)
print(y)

tf.Tensor(
[5 0 4 1 9 2 1 3 1 4 3 5 3 6 1 7 2 8 6 9 4 0 9 1 1 2 4 3 2 7 3 8 6 9 0 5 6
 0 7 6 1 8 7 9 3 9 8 5 9 3 3 0 7 4 9 8 0 9 4 1 4 4 6 0 4 5 6 1 0 0 1 7 1 6
 3 0 2 1 1 7 9 0 2 6 7 8 3 9 0 4 6 7 4 6 8 0 7 8 3 1 5 7 1 7 1 1 6 3 0 2 9
 3 1 1 0 4 9 2 0 0 2 0 2 7 1 8 6 4], shape=(128,), dtype=int64)
tf.Tensor(
[5 0 4 1 9 2 1 3 1 4 3 5 3 6 1 7 2 8 6 9 4 0 9 1 1 2 4 3 2 7 3 8 6 9 0 5 6
 0 7 6 1 8 7 9 3 9 8 5 9 3 3 0 7 4 9 8 0 9 4 1 4 4 6 0 4 5 6 1 0 0 1 7 1 6
 3 0 2 1 1 7 9 0 2 6 7 8 3 9 0 4 6 7 4 6 8 0 7 8 3 1 5 7 1 7 1 1 6 3 0 2 9
 3 1 1 0 4 9 2 0 0 2 0 2 7 1 8 6 4], shape=(128,), dtype=int64)
